In [1]:
%load_ext autoreload
%autoreload 2
import os, sys, json
os.environ['HF_HOME'] = '/home/jovyan/shares/SR003.nfs2/.cache/huggingface'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

In [89]:
import torch
import torch.backends.cudnn as cudnn
from PIL import Image
import numpy as np
import yaml, json
import pandas as pd
import random
import math
from torch import nn
from torchvision import transforms
from transformers import BertTokenizer
from tqdm import tqdm

In [3]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
cudnn.benchmark = True

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

---

### mPLUG-Large

In [63]:
!git clone https://github.com/Yushi-Hu/tifa.git

Cloning into 'tifa'...
remote: Enumerating objects: 271, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 271 (delta 73), reused 67 (delta 67), pack-reused 189 (from 1)
Receiving objects: 100% (271/271), 5.97 MiB | 8.33 MiB/s, done.
Resolving deltas: 100% (143/143), done.


Будем работать с задачей zeroshot VQA. Загружаем large версию модели mPLUG. Воспользуемся реализацией VQA через метчинг логитов с вариантами ответа из бенчмарка TIFA.

In [5]:
sys.path.append('tifa')
from tifascore import VQAModel
model = VQAModel("mplug-large")

/home/jovyan/.mlspace/envs/crave/lib/python3.10/site-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/home/jovyan/.mlspace/envs/crave/lib/python3.10/site-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


[2025-05-31 11:50:30,579] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/jovyan/.mlspace/envs/crave/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/jovyan/.mlspace/envs/crave/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


Loading mplug-large...


2025-05-31 11:50:38,122 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0


2025-05-31 11:50:41,174 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2025-05-31 11:50:41,517 - modelscope - INFO - initiate model from /home/jovyan/.cache/modelscope/hub/models/damo/mplug_visual-question-answering_coco_large_en
2025-05-31 11:50:41,518 - modelscope - INFO - initiate model from location /home/jovyan/.cache/modelscope/hub/models/damo/mplug_visual-question-answering_coco_large_en.
2025-05-31 11:50:41,520 - modelscope - INFO - initialize model from /home/jovyan/.cache/modelscope/hub/models/damo/mplug_visual-question-answering_coco_large_en
/home/jovyan/.mlspace/envs/crave/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1706: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
/home/jovyan/.mlspace/envs/crave/lib/python3.10/site-packages/modelscope/models/mul

load checkpoint from /home/jovyan/.cache/modelscope/hub/models/damo/mplug_visual-question-answering_coco_large_en/pytorch_model.bin
<All keys matched successfully>
Finish loading mplug-large


/home/jovyan/.mlspace/envs/crave/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Using SBERT on GPU


### LLaVA 1.5 HD

In [5]:
from transformers import AutoProcessor, LlavaForConditionalGeneration

In [77]:
def create_prompt(text: str) -> list:
    conversation = [
        {
          "role": "system",
          "content": [
              {
                  "type": "text", "text": "Your task is to answer in the format of a visual question answer."\
                                       "If the question requires a 'yes' or 'no' answer, the answer should contain only 'yes' or 'no'."\
                                       "If the question requires a choice of answer options ('A' or 'B'), the answer should contain only the letter 'A' or 'B'."
              }
          ]
        },
        {
          "role": "user",
          "content": [
              {"type": "text", "text": text},
              {"type": "image"},
            ],
        },
    ]
    return conversation

In [7]:
model_id = "llava-hf/llava-1.5-13b-hf"
model = LlavaForConditionalGeneration.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True, 
).to('cuda')
processor = AutoProcessor.from_pretrained(model_id)

Loading checkpoint shards: 100%|██████████| 6/6 [00:03<00:00,  1.60it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


### Evaluation

In [8]:
from datasets import load_dataset
# датасет VQA
dataset = load_dataset("BaiqiL/NaturalBench")

In [9]:
dataset = dataset['train']

In [30]:
import time
def evaluate_mplug(dataset, model):
    result = 0
    results_all = []
    for d in tqdm(dataset):
        sample_res = 0
        question_0 = d['Question_0']
        question_1 = d['Question_1']
        image_0 = d['Image_0']
        image_1 = d['Image_1']
        image_0.save('im0.png')
        image_1.save('im1.png')
        ans_i0_q0 = d['Image_0_Question_0']
        false_i0_q0 = 'yes' if ans_i0_q0 == 'no' else 'no'
        false_i0_q0 = 'A' if ans_i0_q0 == 'B' else 'B'
        
        ans_i1_q0 = d['Image_1_Question_0']
        false_i1_q0 = 'yes' if ans_i1_q0 == 'no' else 'no'
        false_i1_q0 = 'A' if ans_i1_q0 == 'B' else 'B'
        
        ans_i0_q1 = d['Image_0_Question_1']
        false_i0_q1 = 'yes' if ans_i0_q1 == 'no' else 'no'
        false_i0_q1 = 'A' if ans_i0_q1 == 'B' else 'B'
        
        ans_i1_q1 = d['Image_1_Question_1']
        false_i1_q1 = 'yes' if ans_i1_q1 == 'no' else 'no'
        false_i1_q1 = 'A' if ans_i1_q1 == 'B' else 'B'
        outp_i0_q0 = model.multiple_choice_vqa("im0.png", question_0, choices=[ans_i0_q0, false_i0_q0])['multiple_choice_answer']
        outp_i0_q1 = model.multiple_choice_vqa("im0.png", question_1, choices=[ans_i0_q1, false_i0_q1])['multiple_choice_answer']
        outp_i1_q0 = model.multiple_choice_vqa("im1.png", question_0, choices=[ans_i1_q0, false_i1_q0])['multiple_choice_answer']
        outp_i1_q1 = model.multiple_choice_vqa("im1.png", question_1, choices=[ans_i1_q1, false_i1_q1])['multiple_choice_answer']
        if outp_i0_q0 == ans_i0_q0 and outp_i1_q1 == ans_i1_q1 and outp_i0_q1 == ans_i0_q1 and outp_i1_q1 == ans_i1_q1:
            sample_res = 1
            result += 1
        results_all.append(sample_res)
    return result / len(dataset), results_all     

In [72]:
import re
def LLaVA_inference(model, text, image):
    prompt = processor.apply_chat_template(create_prompt(text), add_generation_prompt=True)
    inputs = processor(images=image, text=prompt, return_tensors='pt').to(0, torch.float16)
    output = model.generate(**inputs, max_new_tokens=200, do_sample=False)
    output = processor.decode(output[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True)
    output = output.strip().lower()
    output = re.split(r'[;:?!,\. ]', output)
    return output

In [78]:
def evaluate_LLaVA(dataset, model):
    result = 0
    results_all = []
    for d in tqdm(dataset):
        sample_res = 0
        question_0 = d['Question_0']
        question_1 = d['Question_1']
        image_0 = d['Image_0']
        image_1 = d['Image_1']
        ans_i0_q0 = d['Image_0_Question_0']
        ans_i1_q0 = d['Image_1_Question_0']
        ans_i0_q1 = d['Image_0_Question_1']
        ans_i1_q1 = d['Image_1_Question_1']
        
        outp_i0_q0 = LLaVA_inference(model, question_0, image_0)
        outp_i1_q0 = LLaVA_inference(model, question_0, image_1)
        outp_i0_q1 = LLaVA_inference(model, question_1, image_0)
        outp_i1_q1 = LLaVA_inference(model, question_1, image_1)
        if ans_i0_q0.lower() in outp_i0_q0 and ans_i1_q1.lower() in outp_i1_q1 \
            and ans_i0_q1.lower() in outp_i0_q1 and ans_i1_q1.lower() in outp_i1_q1:
            sample_res = 1
            result += 1
            # print(result)
        results_all.append(sample_res)
    return result / len(dataset), results_all    

In [ ]:
result_mplug, results_mplug_all = evaluate_mplug(dataset, model)

In [41]:
pd.to_pickle(result_mplug, 'result_mplug.pkl')

In [ ]:
result_LLaVA, results_LLaVA_all = evaluate_LLaVA(dataset, model)

In [65]:
result_mplug = pd.read_pickle('result_mplug.pkl')

In [95]:
print('mPLUG-large results: ', round(result_mplug, 2))

mPLUG-large results:  0.77


In [96]:
print('LLaVA 1.5 (Vicuna-13b) results: ', round(result_LLaVA, 2))

LLaVA 1.5 (Vicuna-13b) results:  0.29


#### Вывод:
- Под задачу VQA лучше обучена модель mPLUG. 